In [100]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import url_changes

from bs4 import BeautifulSoup
import requests

import pandas as pd
import time

##### Ouverture du site booking.com avec Selenium

In [101]:
driver = webdriver.Chrome()

driver.get("https://booking.com")

data = []

current_url = driver.current_url

##### Acceptation des cookies

In [102]:
accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
accept_button.click()

##### Recherche des hébergements sur Paris

- Récupération de la date du jour pour la date d'hébergement

In [103]:
import locale
from datetime import datetime, timedelta

locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

today = datetime.today()
formatted_date = today.strftime("%d %B %Y")

formatted_date = formatted_date.replace(formatted_date.split()[1], formatted_date.split()[1].capitalize())

tomorrow = today + timedelta(days=1)
formatted_tomorrow = tomorrow.strftime("%d %B %Y")

formatted_tomorrow = formatted_tomorrow.replace(formatted_tomorrow.split()[1], formatted_tomorrow.split()[1].capitalize())

print(formatted_date)
print(formatted_tomorrow)

18 Novembre 2024
19 Novembre 2024


- Recherche des hébergements sur Paris pour la date du jour

In [104]:
search_box = WebDriverWait(driver, 10).until(
   EC.presence_of_element_located((By.NAME, "ss"))
)

search_box.send_keys("Paris")

search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//button[.//span[text()='Rechercher']]")
    )
)

date_button = WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//button[@data-testid='date-display-field-start']")
    )
)

date_button.click()

driver.find_element(By.XPATH, f"//span[@aria-label='{formatted_date}']").click()
driver.find_element(By.XPATH, f"//span[@aria-label='{formatted_tomorrow}']").click()

search_button.click()

WebDriverWait(driver, 10).until(url_changes(current_url))

True

##### Fermeture de la pop-up

In [105]:
pop_up_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//div[@class='eb33ef7c47']"))
)

close_button = driver.find_element(By.CSS_SELECTOR, "div.abcc616ec7 button[aria-label='Ignorer les infos relatives à la connexion']")
close_button.click()

##### Affichage de tous les hébergements disponibles

In [15]:
buttonNotLoaded = True

while buttonNotLoaded:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        load_more_button = driver.find_element(By.CSS_SELECTOR, "div[data-results-container='1'] > :last-child > button")
        print("Stop scrolling!")
        buttonNotLoaded = False
    except:
        print("Keep scrolling...")

while True:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        load_more_button = driver.find_element(By.XPATH, "//button[.//span[text()='Afficher plus de résultats']]")

        load_more_button.click()
        time.sleep(1)
    except Exception as e:
        print("Erreur : ", e)
        break


Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Stop scrolling!
Erreur :  Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[.//span[text()='Afficher plus de résultats']]"}
  (Session info: chrome=131.0.6778.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766D42775+28773]
	(No symbol) [0x00007FF766CAAFB0]
	(No symbol) [0x00007FF766B4552A]
	(No symbol) [0x00007FF766B98EAE]
	(No symbol) [0x00007FF766B9919C]
	(No symbol) [0x00007FF766BE23F7]
	(No symbol) [0x00007FF766BBEFDF]
	(No symbol) [0x00007FF766BDF1A4]
	(No symbol) [0x00007FF766BBED43]
	(No symbol) [0x00007FF766B8A548

##### Récupération des hébergements

In [106]:
property_cards = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, "//div[@data-testid='property-card']"))
)

In [107]:
i=0

for card in property_cards:
    i+=1

i

26

Parcours de chaque hebergements et recuperations des notes par catégories

In [ ]:
list_links = []
for card in property_cards:
    link = card.find_element(By.CSS_SELECTOR, "a[data-testid='title-link']").get_attribute("href")
    title = card.find_element(By.CSS_SELECTOR, "div[data-testid='title']").text
    arrondissement = card.find_element(By.CSS_SELECTOR, "span[data-testid='address']").text
    tarif_text = card.find_element(By.CSS_SELECTOR, "span[data-testid='price-and-discounted-price']").text
    tarif_text = tarif_text.replace("€", "")
    tarif = int(tarif_text)
    print(tarif)
    list_links.append([title, link, arrondissement, tarif])

for hotel_title, link, arrondissement, tarif in list_links:
    try:

        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            adresse_element = soup.select_one("span.hp_address_subtitle")
            adresse = adresse_element.get_text(strip=True) if adresse_element else ""
        except Exception as e:
            print(f"Erreur lors de l'extraction de l'adresse pour {hotel_title}: {e}")
            adresse = ""


        try:
            description_elements = soup.select("[data-testid='property-description']")
            description = " ".join([element.get_text(strip=True) for element in description_elements])
        except Exception as e:
            print(f"Erreur lors de l'extraction de la description pour {hotel_title}: {e}")
            description = ""

        try:
            points_fort_elements = soup.select("[data-testid='property-most-popular-facilities-wrapper'] ul li")
            points_fort = [points_fort.get_text(strip=True) for points_fort in points_fort_elements]
        except Exception as e:
            print(f"Erreur lors de l'extraction des points forts pour {hotel_title}: {e}")
            points_fort = []

        reviews = soup.select("div[data-testid='review-subscore']")
        notes = []

        for review in reviews:
            title = review.select_one("span.be887614c2")
            score = review.select_one("div.ccb65902b2.bdc1ea4a28")
            if title and score:
                notes.append({
                    "title_text": title.get_text(strip=True),
                    "score_text": score.get_text(strip=True)
                })

 
        comments = soup.select("li.b0932df2e7")
        hotel_comments = []

        for comment in comments:
            try:
                name = comment.select_one("div.a3332d346a").get_text(strip=True)
                comment_text = comment.select_one("div.a53cbfa6de").get_text(strip=True)
                hotel_comments.append({
                    'Nom': name,
                    'Commentaire': comment_text
                })
            except AttributeError as e:
                print(f"Erreur lors de l'extraction d'un commentaire pour {hotel_title}: {e}")


        data.append({
            "titre": hotel_title,
            "lien_hotel": link,
            "arrondissement": arrondissement,
            "adresse": adresse,
            "description": description,
            "points_fort": points_fort,
            "notes": notes,
            "hotel_comments": hotel_comments,
            "tarif": tarif
        })

    except requests.RequestException as e:
        print(f"Erreur de téléchargement pour {link}: {e}")


427
471
331
72
225
179
260
185
168
88
297
131
65
99
225
170
402
406
138
159
153
143
255
120
181
229
Erreur lors de l'extraction d'un commentaire pour Pullman Paris Tour Eiffel: 'NoneType' object has no attribute 'get_text'
Erreur lors de l'extraction d'un commentaire pour Hôtel National Des Arts et Métiers: 'NoneType' object has no attribute 'get_text'
Erreur lors de l'extraction d'un commentaire pour Paris j'Adore Hotel & Spa: 'NoneType' object has no attribute 'get_text'
Erreur lors de l'extraction d'un commentaire pour The ReMIX Hotel: 'NoneType' object has no attribute 'get_text'
Erreur lors de l'extraction d'un commentaire pour La Finca Hôtel & Spa: 'NoneType' object has no attribute 'get_text'


##### Fermeture de la page avec Selenium

In [ ]:
#driver.quit()

##### Création du dataframe et ajout au fichier csv

In [109]:
df = pd.DataFrame(data, columns=["titre", "lien_hotel","arrondissement","adresse","description","points_fort","notes", "hotel_comments", "tarif"])
df.to_csv("booking.csv", index=False)

In [110]:
df

,titre,lien_hotel,arrondissement,adresse,description,points_fort,notes,hotel_comments,tarif
0,Manolita Paris,https://www.booking.com/hotel/fr/manolita.fr.h...,"18e arr., Paris","1 rue Lepic, 18e arr., 75018 Paris, France","Situé à Paris, à 500 mètres de la station de m...","[Connexion Wi-Fi gratuite, Chambres familiales...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Elisabeth', 'Commentaire': '«J’ai be...",427
1,Pullman Paris Tour Eiffel,https://www.booking.com/hotel/fr/tour-eiffel.f...,"15e arr., Paris","18 Avenue De Suffren, Entrée au 22 rue Jean Re...",L’hôtel 4 étoiles Pullman Paris Tour Eiffel es...,"[Parking sur place, Connexion Wi-Fi gratuite, ...","[{'title_text': 'Personnel', 'score_text': '8,...","[{'Nom': 'Kevin', 'Commentaire': '«Le personne...",471
2,Le Cocon des Sablons - Tour Eiffel Champs Elysées,https://www.booking.com/hotel/fr/beautiful-6pp...,"16e arr., Paris","Appartement 613 14 Rue des Sablons, 16e arr., ...","Situé à Paris, à 1,9 km du palais des congrès ...","[Parking, Connexion Wi-Fi gratuite, Chambres f...","[{'title_text': 'Personnel', 'score_text': '10...",[],331
3,Hotel Aix Europe,https://www.booking.com/hotel/fr/aix-europe.fr...,"10e arr., Paris","4 Rue d'Aix, 10e arr., 75010 Paris, France",Doté d'une connexion Wi-Fi gratuite dans l'ens...,"[Connexion Wi-Fi gratuite, Chambres non-fumeur...","[{'title_text': 'Personnel', 'score_text': '8,...",[],72
4,Secret de Paris - Hotel & Spa,https://www.booking.com/hotel/fr/secret-de-par...,"9e arr., Paris","2 Rue De Parme, 9e arr., 75009 Paris, France",Le Secret de Paris - Hotel & Spa est situé à P...,"[Connexion Wi-Fi gratuite, Spa et centre de bi...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Jean-marc', 'Commentaire': '«Le pers...",225
5,Victoire,https://www.booking.com/hotel/fr/victoire-pari...,"9e arr., Paris","17 Rue de la Victoire, 9e arr., 75009 Paris, F...",Idéalement situé dans le 9ème arrondissement. ...,"[Connexion Wi-Fi gratuite, Chambres familiales...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Charlotte', 'Commentaire': '«Franche...",179
6,Hôtel Elysées Régencia,https://www.booking.com/hotel/fr/elyseesregenc...,"16e arr., Paris","41, Avenue Marceau, 16e arr., 75016 Paris, France",L’Hôtel Elysées Régencia est un établissement ...,"[Parking privé, Connexion Wi-Fi gratuite, Cham...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Anthony', 'Commentaire': '«Personnel...",260
7,Hotel Odeon Saint-Germain,https://www.booking.com/hotel/fr/lodeon.fr.htm...,"6e arr., Paris","13, rue Saint Sulpice, 6e arr., 75006 Paris, F...",Situé dans un quartier animé du centre de Pari...,"[Connexion Wi-Fi gratuite, Chambres familiales...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Jerome', 'Commentaire': '«La disponi...",185
8,Hotel de L'Esperance,https://www.booking.com/hotel/fr/de-l-39-esper...,"5e arr., Paris","15 RUE PASCAL, 5e arr., 75005 Paris, France",Situé à 2 minutes à pied de la très animée rue...,"[Connexion Wi-Fi gratuite, Chambres familiales...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Laure', 'Commentaire': '«La design, ...",168
9,Parc Hotel,https://www.booking.com/hotel/fr/parc.fr.html?...,"14e arr., Paris","60 rue Beaunier, 14e arr., 75014 Paris, France",Le Parc Hotel se trouve dans un quartier paisi...,"[Connexion Wi-Fi gratuite, Chambres familiales...","[{'title_text': 'Personnel', 'score_text': '8,...","[{'Nom': 'Nadjet', 'Commentaire': '«L’emplacem...",88


##### Ajout à la base de données (SQLite & MongoDB)

In [43]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["booking_db"]
collection = db["hebergements"]

collection.insert_many(df.to_dict(orient='records'))

InsertManyResult([ObjectId('673b19a60e175319e448ae3b'), ObjectId('673b19a60e175319e448ae3c'), ObjectId('673b19a60e175319e448ae3d'), ObjectId('673b19a60e175319e448ae3e'), ObjectId('673b19a60e175319e448ae3f'), ObjectId('673b19a60e175319e448ae40'), ObjectId('673b19a60e175319e448ae41'), ObjectId('673b19a60e175319e448ae42'), ObjectId('673b19a60e175319e448ae43'), ObjectId('673b19a60e175319e448ae44'), ObjectId('673b19a60e175319e448ae45'), ObjectId('673b19a60e175319e448ae46'), ObjectId('673b19a60e175319e448ae47'), ObjectId('673b19a60e175319e448ae48'), ObjectId('673b19a60e175319e448ae49'), ObjectId('673b19a60e175319e448ae4a'), ObjectId('673b19a60e175319e448ae4b'), ObjectId('673b19a60e175319e448ae4c'), ObjectId('673b19a60e175319e448ae4d'), ObjectId('673b19a60e175319e448ae4e'), ObjectId('673b19a60e175319e448ae4f'), ObjectId('673b19a60e175319e448ae50'), ObjectId('673b19a60e175319e448ae51'), ObjectId('673b19a60e175319e448ae52'), ObjectId('673b19a60e175319e448ae53'), ObjectId('673b19a60e175319e448ae